In [ ]:
# !pip install praw

In [48]:
import requests
import requests.auth
import pandas as pd
from newspaper import Article 
import praw
import json
import pyimgflip
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR

In [29]:
with open('config.json') as f:
    data = json.load(f)
reddit_cred = data['Reddit']
watson_cred = data['Watson']
newspaper_cred = data['News']
sql_cred = data['SQL']

In [33]:
print(sql_cred['host'])

35.237.95.123


In [39]:
reddit = praw.Reddit(**reddit_cred)

In [40]:
print(reddit.user.me())

jaisal1024


In [31]:
list_ = []
for submission in reddit.subreddit('news').controversial(time_filter = 'week', limit=25):
    submission.comment_sort = 'top'
    dict_ = {
            "Title" : submission.title,
            "url" : submission.url, 
            "permalink" : submission.permalink, 
            "Comments" : submission.comments
        }
    list_.append(dict_)
df = pd.DataFrame(list_)
df

,Comments,Title,permalink,url
0,"(ealf87z, ealfcx5, ealpyzz, ealn2m7, ealvovd, ...",Mexico asks U.S. to investigate use of tear ga...,/r/news/comments/a0yqgr/mexico_asks_us_to_inve...,https://www.reuters.com/article/us-usa-immigra...
1,"(ealn2qt, ealslr5, ealqybu, ealopo5, ealogha, ...",California bar shooter had enough ammo to kill...,/r/news/comments/a0zn0a/california_bar_shooter...,https://apnews.com/412a730295ce4b959bb1fc9b08a...
2,"(ean7k8f, ean8n2e, ean7247, eanac9n, eapvqy5)",Restaurant chain bans mobile phones.,/r/news/comments/a170xc/restaurant_chain_bans_...,https://www.bbc.co.uk/news/amp/business-46369736
3,"(eav9y2c, eauflrq, easviiq, eat0wj4)","Menial Tasks, Slurs and Swastikas: Many Black ...",/r/news/comments/a1uvzj/menial_tasks_slurs_and...,https://www.nytimes.com/2018/11/30/business/te...
4,"(eaoo0u8, eaonagi, eaozqwz, eaomyvk, eaon3jl, ...",Chance US missionary ‘shot dead’ on North Sent...,/r/news/comments/a1cnol/chance_us_missionary_s...,https://www.news.com.au/travel/travel-updates/...
5,"(easbl66, eas4tha, eas5q18, eas9f76, easixgh, ...",Christian Artists Could Face Jail in AZ for Re...,/r/news/comments/a1rkjm/christian_artists_coul...,http://insider.foxnews.com/2018/11/29/christia...
6,"(eap1u03, eap5fbp, eaq2gui, eapt8ar, eaq4lmd, ...",No end set for immigrant kids' internment at '...,/r/news/comments/a1e763/no_end_set_for_immigra...,http://www.msnbc.com/rachel-maddow/watch/no-en...
7,"(eany97c, eanzcgu, eaod7hn, eao0c2o, eaohzlu, ...",Man who attacked Muslim woman in hospital was ...,/r/news/comments/a19q2q/man_who_attacked_musli...,https://m.metrotimes.com/news-hits/archives/20...
8,"(eawf5w3, eawgten, eawfqjd, eawwbmk, eaxa7lt, ...",Las Vegas man blames flu shot for triggering r...,/r/news/comments/a2a464/las_vegas_man_blames_f...,https://news3lv.com/news/local/las-vegas-man-b...
9,"(eaq6b4g, eaq95le, ear2pk5, ear2ppy, ear2q9q, ...",One-third of migrants in caravan are being tre...,/r/news/comments/a1ippa/onethird_of_migrants_i...,https://dmlnews.com/breaking-one-third-migrant...


In [44]:
def extractEntitiesFromUrl(url):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    
    watson_options = {
      "url": url,
      "features": {
        "entities": {
          "sentiment": True,
          "relevance": True,   
        }
      }
    }
    
    username = watson_cred["username"]
    password = watson_cred["password"]

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    data=resp.json()
    # create and return a dictionary for each entity with entity name, url, source, relevance and sentiment score as keys
    entities_list=[]
    for entity in data["entities"]:
        entity_dict={}
        entity_dict["entity"]=entity["text"]
        entity_dict["relevance"]=entity["relevance"]
        entity_dict["sentiment"]=entity["sentiment"]["score"]
        entity_dict["entity"]=entity["text"]
        entities_list.append(entity_dict)
        
    
    return entities_list

def extractEntitiesFromComment(comment):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    
    watson_options = {
      "text": comment,
      "features": {
        "entities": {
          "sentiment": True,
          "relevance": True,
          "emotion" :True
        }
      }
    }
    
    username = watson_cred["username"]
    password = watson_cred["password"]

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    data=resp.json()
    # create and return a dictionary for each entity with entity name, url, source, relevance and sentiment score as keys
    entities_list=[]
    for entity in data["entities"]:
        entity_dict={}
        entity_dict["entity"]=entity["text"]
        entity_dict["relevance"]=entity["relevance"]
        entity_dict["sentiment"]=entity["sentiment"]["score"]
        entity_dict["emotion"]=entity["emotion"]
        entities_list.append(entity_dict)
        
    return entities_list

In [45]:
# entities= [extractEntitiesFromUrl(url) for url in df['url']]
entities = extractEntitiesFromUrl(df['url'][1])
print(df['url'][1])
df_article_entities= pd.DataFrame(entities)
df_article_entities

https://apnews.com/412a730295ce4b959bb1fc9b08a24d04


,entity,relevance,sentiment
0,Ian David Long,0.937235,-0.375068
1,Bill Ayub,0.402139,-0.270387
2,Thousand Oaks,0.397352,-0.654372
3,FBI,0.319592,0.000000
4,Ventura County,0.316953,0.000000
5,Los Angeles,0.316133,-0.314683
6,Southern California,0.291708,-0.369746
7,Calif.,0.275339,0.263084
8,Lee Myers,0.273416,-0.393153
9,Afghanistan,0.258392,-0.813386


In [24]:
comment=df['Comments'][0][0].body
print(comment)
df_comment_entities= pd.DataFrame(extractEntitiesFromComment(comment))
df_comment_entities

Which time?  It’s been used something like 100
times since 2008?  I would be much more concerned about the use of rubber bullets.  Very curious if the President brings up the Mexican military incursions into Texas multiple times over the last decade.


,emotion,entity,relevance,sentiment
0,"{'sadness': 0.607323, 'joy': 0.233274, 'fear':...",President,0.880874,-0.707948
1,"{'sadness': 0.607323, 'joy': 0.233274, 'fear':...",Texas,0.685076,-0.707948


In [32]:
#HOW TO PARSE COMMEMNTS: https://praw.readthedocs.io/en/latest/code_overview/models/comment.html
comment_1st_article = comment=df['Comments'][0]
for comment in comment_1st_article:  
    print(comment.score)

94
75
64
55
53
48
26
10
6
1
1
1
0
-2
-4


In [ ]:
#we could search through comments for entities from article to make specific memes about... using regex 

In [ ]:
meme_py = pyimgflip.Imgflip()
memes = meme_py.get_memes()

df_memes_before = '''
SELECT * FROM MemeNews.Meme_Photos
'''
df_memes_del = '''
SELECT * FROM MemeNews.Deleted_Meme_Photos
'''
curr_memes = pd.read_sql(df_memes_before, engine)['id'].tolist() + pd.read_sql(df_memes_del, engine)['id'].tolist()

#fields in meme: id, name, url, width, height
emotions = ["Anger", "Disgust", "Fear", "Joy", "Sadness"]
df_memes = pd.DataFrame(columns = ["id", "name", "url", "width", "height", "emotion"])
for i in range(len(memes)): 
    if (memes[i].id not in curr_memes):
        print(memes[i].url)
        obj = {
                "id": memes[i].id,
                "name": memes[i].name, 
                "url": memes[i].url, 
                "width": memes[i].width, 
                "height": memes[i].height, 
                "emotion": emotion
            }
        temp  = pd.DataFrame(obj, index = [i])
        delete = input('Enter del to delete, k to keep')
        if (delete != 'del'): 
            emotion = input("Enter an Emotion/s")
            while (emotion not in emotions): 
                print("The Emotions accepted are: ", emotions, "\n")
                emotion = input("Enter an Emotion/s")
            temp.to_sql('Meme_Photos', con = engine, if_exists='append', dtype={'None':VARCHAR(5)})
        else: 
            temp.to_sql('Deleted_Meme_Photos', con = engine, if_exists='append', dtype={'None':VARCHAR(5)})
            
            
            
            
            

https://i.imgflip.com/9ehk.jpg
Enter del to delete, k to keepdel
https://i.imgflip.com/22bdq6.jpg
Enter del to delete, k to keepdel
https://i.imgflip.com/1h7in3.jpg
Enter del to delete, k to keepJoy
Enter an Emotion/sJoy
https://i.imgflip.com/2fm6x.jpg
Enter del to delete, k to keepk
Enter an Emotion/sSadness
https://i.imgflip.com/1yxkcp.jpg
Enter del to delete, k to keepk
Enter an Emotion/sJoy
https://i.imgflip.com/1o00in.jpg
Enter del to delete, k to keepdel
https://i.imgflip.com/1bij.jpg
Enter del to delete, k to keepk
Enter an Emotion/sAnger
https://i.imgflip.com/m78d.jpg
Enter del to delete, k to keepdel
https://i.imgflip.com/1w7ygt.jpg
Enter del to delete, k to keepk
Enter an Emotion/sJoy
https://i.imgflip.com/26am.jpg
Enter del to delete, k to keepk
Enter an Emotion/sDisgust
https://i.imgflip.com/gk5el.jpg
Enter del to delete, k to keepk
Enter an Emotion/sDisgust
https://i.imgflip.com/2896ro.jpg
Enter del to delete, k to keepk
Enter an Emotion/sAnger
https://i.imgflip.com/1bh8.j

In [45]:
# db_name = 'MemeNews'
# # Run a query to create a database that will hold the data
# create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)
# # Create a database
# engine.execute(create_db_query)
conn_string = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = sql_cred["host"], 
    user = sql_cred["user"],
    password = sql_cred["password"], 
    db = 'MemeNews')
engine = create_engine(conn_string)

In [62]:
df_memes.to_sql('Meme_Photos', con = engine, if_exists='append', dtype={'None':VARCHAR(5)})

In [63]:
df_memes = pd.read_sql('SELECT * FROM MemeNews.Meme_Photos', engine)
df_memes

,index,emotion,height,id,name,url,width
0,0,Disgust,800,112126428,Distracted Boyfriend,https://i.imgflip.com/1ur9b0.jpg,1200
1,1,Joy,1893,155067746,Surprised Pikachu,https://i.imgflip.com/2kbn1e.jpg,1893
2,2,Fear,908,87743020,Two Buttons,https://i.imgflip.com/1g8my4.jpg,600
3,3,Disgust,353,102156234,Mocking Spongebob,https://i.imgflip.com/1otk96.jpg,502
